In [1]:
"""
Pipe line to process simulation data and extract features for the inverse modelling task, and save them for future use.
The purpose is to cut down on the time it takes to process the data and extract features.
"""

from pathlib import Path
import json
import pandas as pd
from inverse_modelling_tfo.data_pipelines.fetal_conc_groups import dan_iccps_pencil1, generate_grouping_from_config
from inverse_modelling_tfo.data import config_based_normalization
from inverse_modelling_tfo.data.intensity_interpolation import (
    interpolate_exp,
    get_interpolate_fit_params,
    exp_piecewise_affine,
)
from inverse_modelling_tfo.data.interpolation_function_zoo import *
from inverse_modelling_tfo.features.build_features import (
    FetalACFeatureBuilder,
    RowCombinationFeatureBuilder,
    TwoColumnOperationFeatureBuilder,
    FetalACbyDCFeatureBuilder,
    LogTransformFeatureBuilder,
    ConcatenateFeatureBuilder,
)
from inverse_modelling_tfo.features.data_transformations import (
    LongToWideIntensityTransformation,
    ToFittingParameterTransformation,
)

# Data Setup
# ==========================================================================================
out_dest = Path(r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/processed_data') / 'sythentic_ppg_base.pkl'
# out_dest = Path(__file__).parent.parent.parent / "data" / "processed_data" / "pulastion_ratio.pkl"
# out_dest = Path(__file__).parent.parent.parent / "data" / "processed_data" / "processed1_max_long_range.pkl"
config_dest = out_dest.with_suffix(".json")

in_src = Path(r'/home/rraiyan/simulations/tfo_sim/data/compiled_intensity/synthetic_ppg_base.pkl')
# in_src = Path(r"/home/rraiyan/simulations/tfo_sim/data/compiled_intensity/weitai_data.pkl")
config_src = in_src.with_suffix(".json")

data = pd.read_pickle(in_src)
config_based_normalization(data, config_src)

In [2]:
rounding_points = 2     # Since we are comparing floats, better to round them up to avoid comparison errors
with config_src.open("r") as file:
    config = json.load(file)

fetal_grouping_map = config["fconc_centers"]  # The key will be strings, needs to be converted to float
maternal_grouping_map = config["mconc_centers"]
fetal_grouping_map = {round(float(k), rounding_points): v for k, v in fetal_grouping_map.items()}
maternal_grouping_map = {round(float(k), rounding_points): v for k, v in maternal_grouping_map.items()}

In [3]:
# Data Processing
# ==========================================================================================
data = data.drop(columns="Uterus Thickness")

# Interpolate intensity to remove noise
# data = interpolate_exp(data, weights=(1, 0.6), interpolation_function=exp_piecewise_affine, break_indices=[4, 12, 20])
# data["Intensity"] = data["Interpolated Intensity"]  # Replace OG intensity with interpolated intensity
# data = data.drop(columns="Interpolated Intensity")  # Cleanup

# Define data transformers
data_transformer = LongToWideIntensityTransformation()
# fitting_param_transformer = ToFittingParameterTransformation()

# Transform data
# fitting_params = fitting_param_transformer.transform(data)
data = data_transformer.transform(data)
labels = data_transformer.get_label_names()
intensity_columns = data_transformer.get_feature_names()

# Cleanup
data.dropna(inplace=True)

# Create fetal conc. grouping column - used for generating the AC component/which rows to choose for pairing
data["FconcCenters"] = data["Fetal Hb Concentration"].round(rounding_points).map(fetal_grouping_map)
data["MconcCenters"] = data["Maternal Hb Concentration"].round(rounding_points).map(maternal_grouping_map)
data.head()

,Maternal Wall Thickness,Maternal Hb Concentration,Maternal Saturation,Fetal Hb Concentration,Fetal Saturation,10_2.0,15_2.0,19_2.0,24_2.0,28_2.0,...,64_1.0,68_1.0,72_1.0,77_1.0,81_1.0,86_1.0,90_1.0,94_1.0,FconcCenters,MconcCenters
0,4.0,10.725,0.92,14.625,0.20,0.00001,5.701968e-07,6.087089e-08,5.395582e-09,8.157247e-10,...,8.603914e-09,2.614557e-09,1.012007e-09,4.631602e-10,2.311671e-10,1.272294e-10,7.359312e-11,4.005679e-11,0,0
1,4.0,10.725,0.92,14.625,0.25,0.00001,5.700981e-07,6.083227e-08,5.384875e-09,8.124142e-10,...,8.780234e-09,2.689747e-09,1.046156e-09,4.798490e-10,2.395564e-10,1.317509e-10,7.610140e-11,4.146858e-11,0,0
2,4.0,10.725,0.92,14.625,0.30,0.00001,5.699999e-07,6.079400e-08,5.374286e-09,8.091566e-10,...,8.970441e-09,2.771331e-09,1.083208e-09,4.979972e-10,2.486737e-10,1.366586e-10,7.881864e-11,4.299891e-11,0,0
3,4.0,10.725,0.92,14.625,0.35,0.00001,5.699030e-07,6.075628e-08,5.363868e-09,8.059658e-10,...,1.836546e-08,4.969840e-09,1.697189e-09,7.050752e-10,3.302936e-10,1.772453e-10,1.017818e-10,5.455551e-11,0,0
4,4.0,10.725,0.92,14.625,0.40,0.00001,5.698067e-07,6.071895e-08,5.353691e-09,8.028244e-10,...,1.855857e-08,5.050962e-09,1.733528e-09,7.224904e-10,3.389644e-10,1.819115e-10,1.043865e-10,5.600888e-11,0,0


In [4]:
labels = labels + ["FconcCenters", "MconcCenters"] # This new column grouping should also be treated as a label
# fitting_params['FconcCenters'] = data['FconcCenters']
fixed_columns = [
    "Maternal Wall Thickness",
    "Maternal Saturation",
    "Fetal Saturation",
    "FconcCenters",
    "MconcCenters"
]
data_test = data.groupby(fixed_columns)
data_test.groups

{(4.0, 0.92, 0.2, 0, 0): [0, 11, 110, 121], (4.0, 0.92, 0.2, 0, 1): [220, 231, 330, 341], (4.0, 0.92, 0.2, 0, 2): [440, 451, 550, 561], (4.0, 0.92, 0.2, 0, 3): [660, 671, 770, 781], (4.0, 0.92, 0.2, 0, 4): [880, 891, 990, 1001], (4.0, 0.92, 0.25, 0, 0): [1, 12, 111, 122], (4.0, 0.92, 0.25, 0, 1): [221, 232, 331, 342], (4.0, 0.92, 0.25, 0, 2): [441, 452, 551, 562], (4.0, 0.92, 0.25, 0, 3): [661, 672, 771, 782], (4.0, 0.92, 0.25, 0, 4): [881, 892, 991, 1002], (4.0, 0.92, 0.3, 0, 0): [2, 13, 112, 123], (4.0, 0.92, 0.3, 0, 1): [222, 233, 332, 343], (4.0, 0.92, 0.3, 0, 2): [442, 453, 552, 563], (4.0, 0.92, 0.3, 0, 3): [662, 673, 772, 783], (4.0, 0.92, 0.3, 0, 4): [882, 893, 992, 1003], (4.0, 0.92, 0.35, 0, 0): [3, 14, 113, 124], (4.0, 0.92, 0.35, 0, 1): [223, 234, 333, 344], (4.0, 0.92, 0.35, 0, 2): [443, 454, 553, 564], (4.0, 0.92, 0.35, 0, 3): [663, 674, 773, 784], (4.0, 0.92, 0.35, 0, 4): [883, 894, 993, 1004], (4.0, 0.92, 0.4, 0, 0): [4, 15, 114, 125], (4.0, 0.92, 0.4, 0, 1): [224, 235,

In [5]:
data.iloc[[220, 231, 330, 341], :]

,Maternal Wall Thickness,Maternal Hb Concentration,Maternal Saturation,Fetal Hb Concentration,Fetal Saturation,10_2.0,15_2.0,19_2.0,24_2.0,28_2.0,...,64_1.0,68_1.0,72_1.0,77_1.0,81_1.0,86_1.0,90_1.0,94_1.0,FconcCenters,MconcCenters
220,4.0,11.94375,0.92,14.625,0.2,0.000009,4.790934e-07,4.986243e-08,4.374501e-09,6.713400e-10,...,2.024587e-08,5.761805e-09,2.050524e-09,8.740227e-10,4.137043e-10,2.212896e-10,1.260632e-10,6.813221e-11,0,1
231,4.0,11.94375,0.92,15.000,0.2,0.000009,4.789019e-07,4.979179e-08,4.354805e-09,6.652981e-10,...,2.009269e-08,5.696190e-09,2.020789e-09,8.596263e-10,4.065572e-10,2.174765e-10,1.239583e-10,6.695464e-11,0,1
330,4.0,12.25000,0.92,14.625,0.2,0.000009,4.588327e-07,4.746192e-08,4.154657e-09,6.401820e-10,...,1.923813e-08,5.488428e-09,1.962845e-09,8.407418e-10,3.995835e-10,2.141918e-10,1.221733e-10,6.605419e-11,0,1
341,4.0,12.25000,0.92,15.000,0.2,0.000009,4.586496e-07,4.739329e-08,4.135606e-09,6.343267e-10,...,1.909002e-08,5.424859e-09,1.934055e-09,8.268170e-10,3.926584e-10,2.104970e-10,1.201337e-10,6.491319e-11,0,1


In [6]:
fb1 = RowCombinationFeatureBuilder(intensity_columns, fixed_columns, ["Fetal Hb Concentration", "Maternal Hb Concentration"], "comb", 4)
data = fb1(data)
data.head()

,10_2.0_1,15_2.0_1,19_2.0_1,24_2.0_1,28_2.0_1,33_2.0_1,37_2.0_1,41_2.0_1,46_2.0_1,50_2.0_1,...,FconcCenters,MconcCenters,Fetal Hb Concentration 1,Maternal Hb Concentration 1,Fetal Hb Concentration 2,Maternal Hb Concentration 2,Fetal Hb Concentration 3,Maternal Hb Concentration 3,Fetal Hb Concentration 4,Maternal Hb Concentration 4
0,0.000010,5.701968e-07,6.087089e-08,5.395582e-09,8.157247e-10,1.618866e-10,5.233837e-11,2.094453e-11,9.338248e-12,4.260469e-12,...,0.0,0.0,14.625,10.72500,15.0,10.72500,14.625,11.00,15.0,11.00
1,0.000009,4.790934e-07,4.986243e-08,4.374501e-09,6.713400e-10,1.376183e-10,4.547764e-11,1.836634e-11,8.227240e-12,3.753678e-12,...,0.0,1.0,14.625,11.94375,15.0,11.94375,14.625,12.25,15.0,12.25
2,0.000008,4.039807e-07,4.104586e-08,3.572873e-09,5.573694e-10,1.178480e-10,3.971264e-11,1.616669e-11,7.270813e-12,3.316537e-12,...,0.0,2.0,14.625,13.16250,15.0,13.16250,14.625,13.50,15.0,13.50
3,0.000007,3.419310e-07,3.395054e-08,2.938391e-09,4.666054e-10,1.015868e-10,3.483185e-11,1.427906e-11,6.443921e-12,2.937982e-12,...,0.0,3.0,14.625,14.38125,15.0,14.38125,14.625,14.75,15.0,14.75
4,0.000006,2.899843e-07,2.819831e-08,2.431789e-09,3.936778e-10,8.808936e-11,3.067024e-11,1.265046e-11,5.726026e-12,2.608977e-12,...,0.0,4.0,14.625,15.60000,15.0,15.60000,14.625,16.00,15.0,16.00


In [7]:
fb1.get_label_names()

['Maternal Wall Thickness',
 'Maternal Saturation',
 'Fetal Saturation',
 'FconcCenters',
 'MconcCenters',
 'Fetal Hb Concentration 1',
 'Maternal Hb Concentration 1',
 'Fetal Hb Concentration 2',
 'Maternal Hb Concentration 2',
 'Fetal Hb Concentration 3',
 'Maternal Hb Concentration 3',
 'Fetal Hb Concentration 4',
 'Maternal Hb Concentration 4']

In [9]:
# Create Config file
# ==========================================================================================
# NOT AUTOGENRATED! MUST BE DONE MANUALLY FOR EACH PIPELINE
config = {
    "labels": fb1.get_label_names(),
    "features": fb1.get_feature_names(),
    "feature_builder_txt": str(fb1),
    "preprocessing_description": "Detector Normalization -> Long to Wide -> Row Combination -> Keep 4 intensity columns -> Maternal + Fetal Pulsation",
    "comments": "Data pipeline intended for synthetic ppg generation. Includes both maternal and fetal pulsation. I1 is the stationary case, I2 is fetal pulsation, I3 is maternal pulsation and I4 is a mix(currently we have no use for it)",
}

# Save data and config
# ==========================================================================================
data.to_pickle(out_dest)

with open(config_dest, "w+", encoding="utf-8") as outfile:
    json.dump(config, outfile)
